### 日志分析工具

这个Notebook实现了一个文件监控和合并工具，可以递归扫描文件夹并合并特定模式的日

In [ ]:
import os
from datetime import datetime

### 定义递归文件处理函数
下面的函数实现了递归扫描和合并文件的核心功能。

In [ ]:
def combine_files_recursive(folder_path='.', output_file='combined_logs.txt', file_pattern='error_block_'):
    """
    递归读取文件夹及其子文件夹下的所有指定文件并合并

    参数:
        folder_path: 要扫描的文件夹路径
        output_file: 输出文件名
        file_pattern: 要查找的文件名模式
    """
    def get_all_files(path):
        """递归获取所有符合条件的文件"""
        all_files = []
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.startswith(file_pattern):
                    all_files.append(os.path.join(root, file))
        return sorted(all_files)  # 按文件路径排序

    def write_file_content(input_file, output):
        """写入单个文件内容"""
        try:
            with open(input_file, 'r', encoding='utf-8') as infile:
                # 写入文件路径作为分隔
                output.write(f"\n文件路径: {input_file}\n")
                output.write("-" * 50 + "\n")
                # 写入文件内容
                output.write(infile.read())
                output.write("\n\n")
                return True
        except Exception as e:
            print(f"处理文件 {input_file} 时出错: {str(e)}")
            return False

    # 获取所有符合条件的文件
    files = get_all_files(folder_path)

    # 写入合并文件
    with open(output_file, 'w', encoding='utf-8') as outfile:
        # 写入文件头
        outfile.write("合并日志文件\n")
        outfile.write("=" * 50 + "\n")
        outfile.write(f"生成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        outfile.write(f"源文件夹: {os.path.abspath(folder_path)}\n")
        outfile.write("=" * 50 + "\n\n")

        # 写入文件内容
        success_count = sum(write_file_content(f, outfile) for f in files)

        # 写入统计信息
        outfile.write("\n" + "=" * 50 + "\n")
        outfile.write(f"处理完成！\n")
        outfile.write(f"总文件数: {len(files)}\n")
        outfile.write(f"成功处理: {success_count}\n")
        outfile.write(f"处理失败: {len(files) - success_count}\n")

    print(f"日志已合并保存到 {os.path.abspath(output_file)}")
    return len(files), success_count

### 定义文件夹监控函数
该函数会持续监控指定文件夹，定期执行文

In [ ]:
def monitor_folder(folder_path='.', output_file='combined_logs.txt', file_pattern='error_block_', interval=60):
    """
    持续监控文件夹，定期合并文件

    参数:
        folder_path: 要监控的文件夹路径
        output_file: 输出文件名
        file_pattern: 要查找的文件名模式
        interval: 检查间隔（秒）
    """
    print(f"开始监控文件夹: {os.path.abspath(folder_path)}")
    print(f"检查间隔: {interval}秒")
    print("按 Ctrl+C 停止监控")

    try:
        while True:
            print(f"\n{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - 开始检查...")
            total, success = combine_files_recursive(folder_path, output_file, file_pattern)
            print(f"发现文件: {total}, 成功处理: {success}")
            time.sleep(interval)

    except KeyboardInterrupt:
        print("\n监控已

### 启动监控
执行以下代码开始监控文件夹（默认每60

In [ ]:
# 开始监控文件夹（每60秒检查一次）
monitor_folder(interval=60)